In [15]:
import numpy as np
import os
import pathlib
import ipykernel
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# Definir tamaño de las imágenes
img_width, img_height = 299, 299
batch_size = 32

# Solicitar al usuario que ingrese el nombre de la carpeta y la emoción correcta
selected_folder = input("Ingrese el nombre de la carpeta a utilizar: ")
correct_emotion = input("Ingrese la emoción correcta: ")

# Ruta completa de la carpeta específica
folder_path = 'datasetv2/train'
specific_folder_path = os.path.join(folder_path, selected_folder)

# Obtener la lista de archivos en la carpeta específica
image_files = [os.path.join(specific_folder_path, file) for file in os.listdir(specific_folder_path) if file.endswith(('jpg', 'jpeg', 'png'))]

# Crear un DataFrame con las rutas de las imágenes
data = pd.DataFrame({'filename': image_files})

# Preparar los generadores de datos
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    data,
    x_col='filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,  # No hay etiquetas
    color_mode='rgb',
    shuffle=False
)

# Cargar el modelo entrenado
model = load_model('model_maxv2.h5')

# Hacer predicciones
test_generator.reset()
predictions = model.predict(test_generator, steps=np.ceil(test_generator.samples / batch_size), verbose=1)

# Convertir las predicciones en etiquetas
predicted_classes = np.argmax(predictions, axis=1)

# Mapear las clases predichas a sus etiquetas
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
predicted_emotions = [emotions[i] for i in predicted_classes]

# Calcular el porcentaje de acierto
correct_predictions = predicted_emotions.count(correct_emotion)
accuracy = (correct_predictions / len(predicted_emotions)) * 100

print(f"Porcentaje de acierto: {accuracy:.2f}%")

Found 436 validated image filenames.
14/14 [==============================] - 8s 296ms/step
Porcentaje de acierto: 52.52%
